In [124]:
import requests
import json
import pandas as pd
import geopandas as gpd
import os
from dotenv import load_dotenv
from tqdm import tqdm

## Identify the variables to call from the ACS

### All Possible ACS Vars

In [89]:
## get a list of all the possible variables

url = "https://api.census.gov/data/2018/acs/acs5/variables.json"

response = requests.get(url)
variables_data = response.json()

variables_list = []

for variable, info in variables_data['variables'].items():
    variables_list.append({
        'Variable': variable,
        'Label': info.get('label', ''),
        'Concept': info.get('concept', ''),
        'Group': info.get('group', '')
    })

# Convert the list of dictionaries into a DataFrame
acs_all_variables = pd.DataFrame(variables_list)

In [ ]:
# save for future reference without calling api again

# acs_all_variables.to_csv("./data/acs/variables/acs5_2018_all_vars.csv", index=False)

### Manually Compiled Vars

In [ ]:
# load csv with my manually compiled variables (probably with errors)
variables = pd.read_csv("./data/acs/variables/compiled_acs_vars.csv",usecols=["Variable Code"])


### Merge 
Merge the complete variable list with my manually compiled one to get all the details directly from the census and identify errors

In [96]:
merged_variables = pd.merge(variables, acs_all_variables, how="left", left_on="Variable Code", right_on="Variable")

merged_variables

,Variable Code,Variable,Label,Concept,Group
0,B03002_003E,B03002_003E,Estimate!!Total!!Not Hispanic or Latino!!White...,HISPANIC OR LATINO ORIGIN BY RACE,B03002
1,B01001_002E,B01001_002E,Estimate!!Total!!Male,SEX BY AGE,B01001
2,B01001_003E,B01001_003E,Estimate!!Total!!Male!!Under 5 years,SEX BY AGE,B01001
3,B19013_001E,B19013_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,B19013
4,B17001_002E,B17001_002E,Estimate!!Total!!Income in the past 12 months ...,POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY...,B17001
...,...,...,...,...,...
310,B25083_001E,B25083_001E,Estimate!!Median value (dollars),MEDIAN VALUE (DOLLARS) FOR MOBILE HOMES,B25083
311,B25084_001E,NaN,NaN,NaN,NaN
312,B25086_001E,B25086_001E,Estimate!!Aggregate price asked (dollars),AGGREGATE PRICE ASKED (DOLLARS),B25086
313,B25087_001E,B25087_001E,Estimate!!Total,MORTGAGE STATUS AND SELECTED MONTHLY OWNER COSTS,B25087


In [110]:
# drop incorrect variables 
merged_variables = merged_variables.dropna(subset=["Variable"])
# drop duplicated variables
merged_variables = merged_variables.drop_duplicates()
# drop duplicate column
merged_variables = merged_variables.drop("Variable Code",axis=1)

In [111]:
merged_variables

,Variable,Label,Concept,Group
0,B03002_003E,Estimate!!Total!!Not Hispanic or Latino!!White...,HISPANIC OR LATINO ORIGIN BY RACE,B03002
1,B01001_002E,Estimate!!Total!!Male,SEX BY AGE,B01001
2,B01001_003E,Estimate!!Total!!Male!!Under 5 years,SEX BY AGE,B01001
3,B19013_001E,Estimate!!Median household income in the past ...,MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS ...,B19013
4,B17001_002E,Estimate!!Total!!Income in the past 12 months ...,POVERTY STATUS IN THE PAST 12 MONTHS BY SEX BY...,B17001
...,...,...,...,...
309,B25082_001E,Estimate!!Aggregate value (dollars),AGGREGATE VALUE (DOLLARS) BY MORTGAGE STATUS,B25082
310,B25083_001E,Estimate!!Median value (dollars),MEDIAN VALUE (DOLLARS) FOR MOBILE HOMES,B25083
312,B25086_001E,Estimate!!Aggregate price asked (dollars),AGGREGATE PRICE ASKED (DOLLARS),B25086
313,B25087_001E,Estimate!!Total,MORTGAGE STATUS AND SELECTED MONTHLY OWNER COSTS,B25087


In [112]:
# save final variable df 

merged_variables.to_csv("./data/acs/variables/acs5_2018_selected_vars.csv", index=False)

## Use API to call data for each state

In [ ]:
load_dotenv()

api_key = os.getenv('CENSUS_API')

variable_list = list(merged_variables["Variable"])

In [ ]:
####################################################################################################

def fetch_vars(year, variables, state, api_key):

    url = f"https://api.census.gov/data/{year}/acs/acs5?get={variables}&for=block%20group:*&in=state:{state}%20county:*%20tract:*&key={api_key}"

    try:
        response = requests.get(url)

        if response.status_code == 200:

            print("Response Code 200")
            return response.json()
        
        else:

            print(f"Failed batch request")
            return None
        
    except requests.exceptions.RequestException as e:
        print(f"Request error for {variable}: {e}")
        return None

####################################################################################################

def merge_dataframes(df_list):

    final_df = df_list[0]

    for df in df_list[1:]:

        final_df = pd.merge(final_df, df, on=["state","county","tract","block group"])

    return final_df

####################################################################################################

def make_geoid(df):

    df["GEOID"] = (
        df["state"].str.zfill(2) +
        df["county"].str.zfill(3) +
        df["tract"].str.zfill(6) +
        df["block group"]
    )


In [180]:
batch_size = 50 
batches = [variable_list[i:i + batch_size] for i in range(0, len(variable_list), batch_size)]

len(batches)

6

In [181]:
md_results = []

for batch in tqdm(batches):

    variables = ",".join(batch)

    data = fetch_vars(year=2018, variables=variables, state="24", api_key=api_key)

    if data:
        df_batch = pd.DataFrame(data[1:], columns=data[0])
        md_results.append(df_batch)

    else:
        print("batch failed")

 17%|█▋        | 1/6 [01:20<06:40, 80.03s/it]

Response Code 200


 33%|███▎      | 2/6 [02:38<05:17, 79.33s/it]

Response Code 200


 50%|█████     | 3/6 [03:59<04:00, 80.10s/it]

...

Response Code 200


 83%|████████▎ | 5/6 [06:38<01:19, 79.65s/it]

Response Code 200


100%|██████████| 6/6 [07:59<00:00, 79.89s/it]

Response Code 200


In [182]:
md_final_df = merge_dataframes(md_results)

make_geoid(md_final_df)

In [ ]:
md_final_df.to_csv("./data/acs/acs5_2018_selected_vars_MD.csv",index=False)

## Old Test Code

In [ ]:
load_dotenv()

api_key = os.getenv('CENSUS_API')
year = 2023
state = "11"
variables = ",".join(list(variables["Var"]))

url = f"https://api.census.gov/data/{year}/acs/acs5?get={variables}&for=block%20group:*&in=state:{state}%20county:*%20tract:*&key={api_key}"


response = requests.get(url)

response.text


'[["B01001_001E","B03002_003E","B01001_002E","B01001_003E","B19013_001E","B17001_002E","B25003_001E","B25003_002E","B25070_001E","B25064_001E","B25091_001E","B15003_022E","B23025_003E","B23025_004E","B18101_002E","B11009_001E","B16001_002E","state","county","tract","block group"],\n["1181","837","503","35","135708",null,"803","436","367","2590","436","398","745","728",null,null,null,"11","001","000101","1"],\n["1079","979","394","0","156440",null,"771","339","432","3501","339","510","855","855",null,null,null,"11","001","000102","1"],\n["1287","1150","706","91","229199",null,"573","533","40","3501","533","221","697","669",null,null,null,"11","001","000102","2"],\n["690","378","338","12","146250",null,"486","340","146","3284","340","276","408","402",null,null,null,"11","001","000102","3"],\n["0","0","0","0","-666666666",null,"0","0","0","-666666666","0","0","0","0",null,null,null,"11","001","000201","1"],\n["2835","1538","1130","0","-666666666",null,"0","0","0","-666666666","0","0","108

In [60]:
test = fetch_var(year=2018, variable='B01001_001E', state="11", api_key=os.getenv('CENSUS_API'))

In [62]:
df = pd.DataFrame(test[1:], columns=test[0])
df


,B01001_001E,state,county,tract,block group
0,857,11,001,006802,1
1,623,11,001,002202,3
2,1900,11,001,000600,4
3,1429,11,001,004201,1
4,1805,11,001,001100,1
...,...,...,...,...,...
445,1394,11,001,008802,4
446,1968,11,001,007903,1
447,3184,11,001,008904,1
448,839,11,001,007401,1


In [39]:
data = response.json()

In [40]:
df = pd.DataFrame(data[1:], columns=data[0])
df

,B01001_001E,B03002_003E,B01001_002E,B01001_003E,B19013_001E,B17001_002E,B25003_001E,B25003_002E,B25070_001E,B25064_001E,...,B15003_022E,B23025_003E,B23025_004E,B18101_002E,B11009_001E,B16001_002E,state,county,tract,block group
0,1181,837,503,35,135708,None,803,436,367,2590,...,398,745,728,None,None,None,11,001,000101,1
1,1079,979,394,0,156440,None,771,339,432,3501,...,510,855,855,None,None,None,11,001,000102,1
2,1287,1150,706,91,229199,None,573,533,40,3501,...,221,697,669,None,None,None,11,001,000102,2
3,690,378,338,12,146250,None,486,340,146,3284,...,276,408,402,None,None,None,11,001,000102,3
4,0,0,0,0,-666666666,None,0,0,0,-666666666,...,0,0,0,None,None,None,11,001,000201,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566,518,326,248,0,84849,None,392,272,120,2333,...,172,443,399,None,None,None,11,001,011002,2
567,1480,165,742,63,117730,None,449,380,69,-666666666,...,228,838,658,None,None,None,11,001,011100,1
568,1786,325,1023,89,-666666666,None,802,426,376,525,...,281,813,716,None,None,None,11,001,011100,2
569,1719,49,859,78,61172,None,604,261,343,-666666666,...,175,684,561,None,None,None,11,001,011100,3


In [24]:
df["GEOID"] = (
    df["state"].str.zfill(2) +
    df["county"].str.zfill(3) +
    df["tract"].str.zfill(6) +
    df["block group"]
)

In [25]:
len(df["GEOID"].unique())

571

In [26]:
df

,NAME,B01003_001E,B01001_001E,B06009_006E,state,county,tract,block group,GEOID
0,Block Group 1; Census Tract 1.01; District of ...,1181,1181,None,11,001,000101,1,110010001011
1,Block Group 1; Census Tract 1.02; District of ...,1079,1079,None,11,001,000102,1,110010001021
2,Block Group 2; Census Tract 1.02; District of ...,1287,1287,None,11,001,000102,2,110010001022
3,Block Group 3; Census Tract 1.02; District of ...,690,690,None,11,001,000102,3,110010001023
4,Block Group 1; Census Tract 2.01; District of ...,0,0,None,11,001,000201,1,110010002011
...,...,...,...,...,...,...,...,...,...
566,Block Group 2; Census Tract 110.02; District o...,518,518,None,11,001,011002,2,110010110022
567,Block Group 1; Census Tract 111; District of C...,1480,1480,None,11,001,011100,1,110010111001
568,Block Group 2; Census Tract 111; District of C...,1786,1786,None,11,001,011100,2,110010111002
569,Block Group 3; Census Tract 111; District of C...,1719,1719,None,11,001,011100,3,110010111003
